# 프롬프트 엔지니어링(Prompt Engineering)

프롬프트 엔지니어링은 단순히 질문을 던지는 것을 넘어, 모델의 작동 원리와 '인컨텍스트 러닝(In-context Learning)' 능력을 활용해 모델의 출력을 제어하는 프로세스이다. 이는 모델의 파라미터(가중치)를 직접 수정하지 않고도 모델의 성능을 특정 태스크에 맞게 조정하는 방법론이다.

**프롬프트의 핵심 구성 요소:**

효과적인 프롬프트는 일반적으로 다음의 4가지 요소를 포함한다.

* **지시문 (Instruction):** 모델이 수행해야 할 구체적인 작업(예: 요약하라, 분류하라, 번역하라 등).
* **문맥 (Context):** 모델이 작업을 더 잘 수행하도록 돕는 배경 정보나 제약 조건.
* **입력 데이터 (Input Data):** 처리가 필요한 실제 데이터.
* **출력 지시자 (Output Indicator):** 결과물의 형식이나 스타일 지정(예: 표로 정리하라, JSON 포맷으로 출력하라 등).

**프롬프트 엔지니어링의 중요성:**

* **성능 최적화:** 같은 모델이라도 프롬프트에 따라 성능 차이가 극심하다. 잘 설계된 프롬프트는 더 작은 모델로도 큰 모델 수준의 결과를 낼 수 있게 한다.
* **비용 효율성:** 불필요한 토큰 사용을 줄이고, 파인튜닝(Fine-tuning)에 비해 적은 비용으로 도메인 특화 작업을 수행할 수 있다.
* **한계 극복:** 모델의 환각 현상을 줄이고 최신 정보를 반영(RAG와 결합 시)하도록 유도할 수 있다.

In [36]:
from openai import OpenAI                 # OpenAI API를 사용하기 위한 클라이언트 클래스
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv('openai_key')
client = OpenAI(api_key = api_key)

### 뉴스 기사 제목 수정

In [37]:
def correct_headline(headline, /, *, model='gpt-4.1-mini', temperature=1, top_p=1, max_completion_tokens=2048):
    response = client.chat.completions.create(                 # OpenAI Chat Completions API 호출
        model=model,                                           # 사용할 모델 지정
        messages=[                                             # 대화 메시지 목록
            {
                "role": "system",                              # 시스템 역할: 모델의 페르소나/규칙 정의
                "content": [
                    {
                        "type": "text",                        # 텍스트 타입 메시지
                        "text": "기자들이 송고한 제목에서 맞춤법/문법/의미/어조등을 고려해 최상의 뉴스제목을 뽑아내는 20년 경력의 뉴스제목교정가이다.\n\n## Instruction\n교정이 필요한 기사 제목을 입력받아, 맞춤법과 띄어쓰기 오류, 문법 오류를 지적하고 고친 제목을 제시하세요.  \n아래 단계로 진행합니다:  \n1. 입력된 기사 제목을 면밀히 분석하여 맞춤법 오류, 띄어쓰기 실수, 문법 오류 등 문제점을 찾아 지적 항목으로 정리합니다.  \n2. 문제점을 모두 고친 교정된 기사 제목을 결과로 제시합니다.  \n3. 교정이 필요한 부분과 수정결과를 교정이유항목에 작성해주세요.\n4. 기사 제목에 오류가 여러 개 있을 경우, 각 오류를 번호를 매겨 명확히 구분하여 지적합니다.\n5. 독자의 관심을 끌수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.\n6. 어조가 지나치게 감정적이거나 부정적이라면, 적절히 중립적 표현을 사용하세요.\n7. 비속어/욕설등이 포함되어 있다면 이를 제거하고, 의미가 전달될수 있는 적절한 표현으로 수정하세요. \n\n## Output Format\n- 원래제목: [송고한 기사제목]\n- 교정제목: [교정한 기사제목]\n- 교정 이유:\n  1. [교정한 부분과 이유]\n  2. [교정한 부분과 이유]\n\n## Examples\n<예시1>  \n입력: \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\"  \n출력:  \n- 원래제목: [송고한 기사제목]\n- 교정제목: \"코로나19 백신 접종률 높이기 위한 대안 마련 시급\"\n- 교정 이유:\n   1. '접종율'은 '접종률'이 맞는 표기입니다.\n   2. '높히기'는 '높이기'로, 맞춤법 오류입니다.\n   3. '대안마련'은 붙여쓰지 않고 '대안 마련'으로 띄어 써야 맞습니다.\n   4. 간결한 어미수정\n\n<예시2>  \n입력: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\"  \n출력:  \n- 원래제목: [송고한 기사제목]\n- 교정제목: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야\"\n- 교정 이유:\n  - 간결한 어미 수정\n"  
                                                                # 제목 교정 전문가 역할 + 상세 규칙 프롬프트
                    }
                ]
            },
            {
                "role": "user",                                # 사용자 역할: 실제 입력 데이터
                "content": [
                    {
                        "type": "text",                        # 텍스트 입력
                        "text": f"입력: {headline}"            # 함수 인자로 받은 기사 제목(헤드라인)
                    }
                ]
            }
        ],
        response_format={"type": "text"},                      # 응답을 순수 텍스트로 받음
        temperature=temperature,                               # 생성 다양성 조절
        max_completion_tokens=max_completion_tokens,           # 최대 응답 토큰 수
        top_p=top_p,                                           # 확률 질량 기반 샘플링
        frequency_penalty=0,                                   # 반복 패널티 없음
        presence_penalty=0,                                    # 새로운 주제 유도 패널티 없음
        store=False                                            # 서버 저장 비활성화
    )
    return response.choices[0].message.content                 # 모델이 생성한 최종 결과 반환


             

/ (positional-only-expression) : /의 왼쪽 매개변수는 위치인자 방식으로만 호출 강제화 (headline이 첫번째 위치에서 필수입력이 강제된다.)
* (keyword-only-expression) : *의 오른쪽 매개변수는 키워드인자 방식으로만 호출 강제화(model=..., temperature=..., 이런식으로 키워드로만 받는다.)

In [38]:
correct_headline(
    "두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네",model="gpt-4.1-mini"
)             

"- 원래제목: 두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네\n- 교정제목: 두쫀쿠 열풍, 무엇이 인기인지 알기 어렵다\n- 교정 이유:\n  1. '존맛탱'은 비속어로 적절하지 않아 '인기'로 순화했습니다.\n  2. '알수가'는 '알 수가'로 띄어 써야 합니다.\n  3. 문장을 간결하고 중립적으로 바꾸기 위해 '뭐가'를 '무엇이'로 수정했습니다.\n  4. 어조가 지나치게 감정적이므로 '알기 어렵다'로 정중하게 표현했습니다.\n  5. 쉼표를 추가해 문장 호흡을 자연스럽게 조정했습니다."

In [39]:
headlines= [
    "주말 성수동거리에는 커플들이 그득그득 하더라",
    "졸라 빡센 작업으로 끼닌 거르기 일쑤인 노동자들의 애환",
    "유튜버들의 생존일기, 쉽지 않은 그들의 여정에 함께헤요~"
]
for headline in headlines:
    output = correct_headline(headline)
    print(output)

- 원래제목: 주말 성수동거리에는 커플들이 그득그득 하더라  
- 교정제목: 주말 성수동 거리에는 커플이 그득했다  
- 교정 이유:  
  1. '성수동거리'는 지명과 '거리'를 띄어 써야 하므로 '성수동 거리'로 수정했습니다.  
  2. '커플들이'는 복수형 주어이지만 '그득하다'와 같이 많은 인원이 밀집한 상태를 표현할 때는 단수형 주어 '커플이'가 더 자연스럽습니다.  
  3. '그득그득 하더라'는 구어체 표현으로, 기사 제목에 부적합하므로 간결하고 공식적인 '그득했다'로 바꾸었습니다.  
  4. 전체적으로 간결하며 전달력이 좋은 어투로 정리했습니다.
- 원래제목: 졸라 빡센 작업으로 끼닌 거르기 일쑤인 노동자들의 애환
- 교정제목: 매우 힘든 작업 때문에 끼니 거르기 잦은 노동자들의 애환
- 교정 이유:  
  1. '졸라'는 비속어로 부적절하여 '매우'로 바꾸어 중립적이고 공적인 표현으로 수정했습니다.  
  2. '빡센' 역시 비속어이며, '힘든'으로 바꾸어 표현을 정제했습니다.  
  3. '끼닌'은 '끼니'의 오타로, 올바른 맞춤법인 '끼니'로 고쳤습니다.  
  4. '거르기' 대신 상황에 맞게 '거르기 잦은'으로 의미를 명확히 했습니다.  
  5. 전반적으로 어조를 중립적이고 간결하게 조정하여 정보 전달에 초점을 맞췄습니다.
- 원래제목: 유튜버들의 생존일기, 쉽지 않은 그들의 여정에 함께헤요~
- 교정제목: 유튜버들의 생존일기, 쉽지 않은 여정 함께해요
- 교정 이유:
  1. '함께헤요'는 '함께해요'로 띄어쓰기와 맞춤법을 바로잡아야 합니다.
  2. '~'는 뉴스 제목에서는 지양하는 것이 좋으며, 간결한 문장 표현을 위해 삭제했습니다.
  3. '그들의'는 중복 표현으로 보이므로 간결하게 '쉽지 않은 여정'으로 수정했습니다.
  4. 어조를 부드럽고 친근하게 유지하면서도 뉴스 제목에 적합하도록 다듬었습니다.


In [40]:
# JSON(dict) 형태로 교정 결과 
import json

def correct_headline_json(headline, /, *, model='gpt-4.1-mini', temperature=1, top_p=1, max_completion_tokens=2048):
    response = client.chat.completions.create(                 # OpenAI Chat Completions API 호출
        model=model,                                           # 사용할 모델 지정
        messages=[                                             # 대화 메시지 목록
            {
                "role": "system",                              # 시스템 역할: 모델의 페르소나/규칙 정의
                "content": [
                    {
                        "type": "text",                        # 텍스트 타입 메시지
                        # Json 출력 포멧과 예시
                        "text": "기자들이 송고한 제목에서 맞춤법/문법/의미/어조등을 고려해 최상의 뉴스제목을 뽑아내는 20년 경력의 뉴스제목교정가이다.\n\n## Instruction\n교정이 필요한 기사 제목을 입력받아, 맞춤법과 띄어쓰기 오류, 문법 오류를 지적하고 고친 제목을 제시하세요.  \n아래 단계로 진행합니다:  \n1. 입력된 기사 제목을 면밀히 분석하여 맞춤법 오류, 띄어쓰기 실수, 문법 오류 등 문제점을 찾아 지적 항목으로 정리합니다.  \n2. 문제점을 모두 고친 교정된 기사 제목을 결과로 제시합니다.  \n3. 교정이 필요한 부분과 수정결과를 교정이유항목에 작성해주세요.\n4. 기사 제목에 오류가 여러 개 있을 경우, 각 오류를 번호를 매겨 명확히 구분하여 지적합니다.\n5. 독자의 관심을 끌수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.\n6. 어조가 지나치게 감정적이거나 부정적이라면, 적절히 중립적 표현을 사용하세요.\n7. 비속어/욕설등이 포함되어 있다면 이를 제거하고, 의미가 전달될수 있는 적절한 표현으로 수정하세요. \n\n## Output Format\n**반드시 json 객체 형식을 준수하세요.**\n\n{{\n  \"original_headline\": <송고한 기사제목>,\n  \"corrected_headline\": <교정한 기사제목>,\n  \"reasons\": [\n     <교정한 부분과 이유>,\n     <교정한 부분과 이유>,\n  ] \n}}\n\n## Examples\n<예시1>  \n입력: \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\"  \n출력:  \n{{\n  \"original_headline\": \"코로나19 백신접종율 높히기 위한 대안마련 필요하다\",\n  \"corrected_headline\": \"코로나19 백신 접종률 높이기 위한 대안 마련 시급\",\n  \"reasons\": [\n    \"'접종율'은 표준어가 아니며 '접종률'이 올바른 표기이다\",\n    \"'높히기'는 맞춤법 오류로 '높이기'로 수정해야 한다\",\n    \"'대안마련'은 띄어 써야 하므로 '대안 마련'으로 수정하였다\",\n    \"기사 제목에 맞게 어미를 간결하게 다듬었다\"\n  ]\n}}\n\n<예시2>  \n입력: \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\"  \n출력:  \n{{\n  \"original_headline\": \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야한다\",\n  \"corrected_headline\": \"정부, 연금개혁 지방화 시대 맞춰 적극 나서야\",\n  \"reasons\": [\n    \"기사 제목의 문체에 맞게 불필요한 어미를 제거해 간결하게 수정하였다\"\n  ]\n}}\n"  
                                                                # 제목 교정 전문가 역할 + 상세 규칙 프롬프트
                    }
                ]
            },
            {
                "role": "user",                                # 사용자 역할: 실제 입력 데이터
                "content": [
                    {
                        "type": "text",                        # 텍스트 입력
                        "text": f"입력: {headline}"            # 함수 인자로 받은 기사 제목(헤드라인)
                    }
                ]
            }
        ],
        response_format={"type": "json_object"},               # 응답을 순수 json으로 받음
        temperature=temperature,                               # 생성 다양성 조절
        max_completion_tokens=max_completion_tokens,           # 최대 응답 토큰 수
        top_p=top_p,                                           # 확률 질량 기반 샘플링
        frequency_penalty=0,                                   # 반복 패널티 없음
        presence_penalty=0,                                    # 새로운 주제 유도 패널티 없음
        store=False                                            # 서버 저장 비활성화
    )
    return json.loads(response.choices[0].message.content)                 # 모델이 생성한 최종 결과 반환


             

In [41]:
# JSON(dict) 형태로 교정 결과 받기
output = correct_headline_json('두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네')
print(output)

{'original_headline': '두쫀쿠 열풍 뭐가 존맛탱이라는 건지 알수가 없네', 'corrected_headline': '두쫀쿠 열풍, 왜 인기인지 알기 어렵다', 'reasons': ["비속어 '존맛탱'을 중립적 표현인 '인기'로 수정하였다", "'알수가'는 띄어쓰기가 잘못되어 '알 수가'로 변경해야 하지만, 문장을 더 자연스럽게 다듬기 위해 전체 문장 구조를 수정하였다", "'건지'는 구어체이며 '왜'로 바꾸어 공식적인 어조에 맞게 표현하였다", "문장 끝의 '-네'는 비격식체이므로 뉴스 제목에 적합한 중립적 표현으로 수정하였다", "'두쫀쿠 열풍 뭐가' 부분에 쉼표를 추가하여 가독성을 높였다"]}


In [42]:
correct_headline = output["corrected_headline"]
correct_headline

'두쫀쿠 열풍, 왜 인기인지 알기 어렵다'

In [43]:
headlines= [
    "주말 성수동거리에는 커플들이 그득그득 하더라",
    "졸라 빡센 작업으로 끼닌 거르기 일쑤인 노동자들의 애환",
    "유튜버들의 생존일기, 쉽지 않은 그들의 여정에 함께헤요~"
]

headlines = [correct_headline_json(headline)['corrected_headline'] for headline in headlines]

for headline in headlines:
    print(headline)


주말 성수동 거리에는 커플들로 가득했다
거친 작업으로 끼니 거르기 일쑤인 노동자들의 애환
유튜버들의 생존 일기, 쉽지 않은 그들의 여정에 함께해요


# 냉털마스터

In [51]:
# 입력 재료만으로 가능한 음식 2가지를 json형태로 추천해주는 함수
def refing_master_json(user_input: str, /, *, model='gpt-4.1-mini', temperature=1, top_p=1, max_completion_tokens=2048):
    response = client.chat.completions.create(                 # OpenAI Chat Completions API 호출
        model=model,                                           # 사용할 모델 지정
        messages=[                                             # 대화 메시지 목록
            {
                "role": "system",                              # 시스템 역할: 모델의 페르소나/규칙 정의
                "content": [
                    {
                        "type": "text",                        # 텍스트 타입 메시지
                        "text": "# Instruction\n사용자가 입력한 냉장고 내 재료 목록만을 사용하여 만들 수 있는 음식 2가지를 추천하세요.  \n반드시 입력된 재료만 활용하며, 기본양념(간장, 소금, 설탕, 설탕, 식초, 후추 등)은 언제든 사용할 수 있다고 가정하세요.  \n입력된 목록에 없는 재료(양념 제외)는 절대 사용하지 말고, 추가 재료 없이 조리 가능한 음식만 선정하십시오.  \n음식 종류는 반드시 서로 비슷하지 않은 2가지여야 하며, 각 음식에 대한 자세한 요리 레시피(조리 순서)를 단계별로 포함하세요.\n\n- 먼저, 입력 재료로 만들 수 있는 음식 종류를 논리적으로 검토한 뒤, 각 음식이 왜 가능한지 간단히 설명해 주세요.\n- reasoning(논리 및 설명)과 conclusion(최종 추천 결과)은 반드시 JSON의 개별 필드로 구분하여 제공하십시오.\n- reasoning이 반드시 먼저, conclusion이 반드시 마지막에 위치해야 합니다.\n- 결론(conclusion)에는 각 음식명과 단계별 레시피를 포함하세요.\n- 반드시 모든 답변을 한글로 작성하세요.\n\n# Steps\n\n1. 입력 재료만 활용 가능한 음식 2가지를 선정하고, 서로 비슷하지 않은지 확인하세요.\n2. reasoning(논리/설명) 필드에: \n    - 해당 재료로 어떤 음식이 가능한지, 그 이유를 간단히 단계별 논리로 설명하세요.\n3. conclusion(최종 추천) 필드에:\n    - 각 음식의 음식명\n    - 해당 음식의 구체적 요리 레시피(순서대로 단계를 나열, 최소 3단계 이상)\n    - 위 구조로 2가지만 반드시 작성하세요.\n\n# Output Format\n\n모든 답변은 아래 JSON 구조로 출력하세요.  \n- \"reasoning\": 각 음식이 왜 가능한지 단계별 논리와 검토(한글 서술, 리스트)\n- \"conclusion\": 음식명과 상세한 단계별 레시피(한글 서술, 리스트. 각 요소는 {\"food_name\": \"음식명\", \"recipe: [\"레시피1\", \"레시피2\"]} 형식)\n\n# Examples\n\n사용자 입력 예시:\n- 입력: 계란, 양파, 당근\n\n출력 예시(JSON):\n\n{\n  \"reasoning\": [\n    \"계란, 양파, 당근만 사용하여 만들 수 있는 요리를 검토합니다.\",\n    \"계란과 채소(양파, 당근)만으로 달걀전이 가능합니다. 채소를 잘게 썰어 계란과 섞어 부치면 완성할 수 있습니다.\",\n    \"계란찜 역시 이 재료로 만들 수 있습니다. 계란을 풀고 다진 채소를 섞은 후, 찜기를 사용해 익히면 완성됩니다.\"\n  ],\n  \"conclusion\": [\n    {\n      \"food_name\": \"달걀전\",\n      \"recipe\": [\n        \"1. 양파와 당근을 잘게 썰어줍니다.\",\n        \"2. 계란을 풀고 썰어둔 양파와 당근, 소금, 후추를 넣고 섞습니다.\",\n        \"3. 달궈진 팬에 기름을 두르고 반죽을 얇게 올린 후, 앞뒤로 노릇하게 부칩니다.\"\n      ]\n    },\n    {\n      \"food_name\": \"계란찜\",\n      \"recipe\": [\n        \"1. 계란을 볼에 넣고 곱게 풀어줍니다.\",\n        \"2. 다진 양파와 당근, 소금, 후추를 계란물에 넣고 섞습니다.\",\n        \"3. 뚝배기나 내열 용기에 재료를 옮겨 담고, 중탕 또는 전자레인지로 익혀 부드럽게 완성합니다.\"\n      ]\n    }\n  ]\n}\n\n(실제 예시는 입력 재료와 음식에 따라 달라지며, 각 음식의 레시피 단계는 3단계 이상, 충분히 구체적으로 작성하십시오.)\n\n# Notes\n\n- 반드시 입력 재료만 사용하고, 음식명 및 조리법 전부 한글로 기입하세요.\n- 각 추천 요리는 서로 다른 종류여야 하며, 각 음식마다 레시피 단계는 구체적이고 논리적으로 작성돼야 합니다.\n- reasoning(논리/설명) → conclusion(최종 추천 및 레시피) 순서는 꼭 지켜야 합니다.\n- 답변 형식은 반드시 JSON이어야 하며, 한글로만 작성하세요.\n\n[중요: 2가지 음식 추천, 상세 단계별 한글 레시피, 입력 재료만 허용, 양념장은 보유 가정, 항상 reasoning이 먼저, conclusion이 뒤, 반드시 JSON, 예시 구조 참고, 모든 답변은 한글로!]"
                                                                # 제목 교정 전문가 역할 + 상세 규칙 프롬프트
                    }
                ]
            },
            {
                "role": "user",                                # 사용자 역할: 실제 입력 데이터
                "content": [
                    {
                        "type": "text",                        # 텍스트 입력
                        "text": f"입력: {user_input}"            # 함수 인자로 받은 기사 제목(헤드라인)
                    }
                ]
            }
        ],
        response_format={"type": "json_object"},               # 응답을 순수 json으로 받음
        temperature=temperature,                               # 생성 다양성 조절
        max_completion_tokens=max_completion_tokens,           # 최대 응답 토큰 수
        top_p=top_p,                                           # 확률 질량 기반 샘플링
        frequency_penalty=0,                                   # 반복 패널티 없음
        presence_penalty=0,                                    # 새로운 주제 유도 패널티 없음
        store=False                                            # 서버 저장 비활성화
    )
    return json.loads(response.choices[0].message.content)                 # 모델이 생성한 최종 결과 반환

In [ ]:
refing_master_json("계란,양파,당근")

{'reasoning': ['계란, 양파, 당근 세 가지 재료만으로 만들 수 있는 요리를 검토합니다.',
  '첫째, 계란과 채소를 활용한 전 형태의 음식이 가능합니다. 양파와 당근을 잘게 썰어 계란과 섞으면 부치는 방식의 달걀전이 완성됩니다.',
  '둘째, 계란찜도 가능한 음식입니다. 계란을 풀고 다진 양파와 당근을 혼합한 후 찜하거나 전자레인지에 익히면 부드러운 계란찜이 됩니다.',
  '이 두 음식은 조리 방식과 최종 형태가 서로 다르므로 서로 비슷하지 않은 음식으로도 적합합니다.'],
 'conclusion': [{'food_name': '달걀전',
   'recipe': ['1. 양파와 당근을 깨끗이 씻고 껍질을 벗긴 뒤, 아주 잘게 다집니다.',
    '2. 계란을 그릇에 깨뜨려 넣고 소금과 후추를 약간 넣어 잘 풀어줍니다.',
    '3. 다진 양파와 당근을 계란물에 넣고 골고루 섞어줍니다.',
    '4. 팬을 중불로 예열하고 기름을 약간 두른 후, 반죽을 한 국자씩 떠넣어 앞뒤로 노릇노릇하게 부칩니다.',
    '5. 완성된 달걀전을 접시에 담아내면 됩니다.']},
  {'food_name': '계란찜',
   'recipe': ['1. 계란을 그릇에 깨뜨려 넣고 소금과 후추로 간을 하여 고루 풀어줍니다.',
    '2. 양파와 당근을 깨끗이 씻고 잘게 다진 후 계란물에 넣어 섞어줍니다.',
    '3. 내열 용기에 혼합물을 부어준 뒤 랩을 덮거나 뚜껑을 덮습니다.',
    '4. 중탕으로 찌거나 전자레인지에 3~4분 정도 돌려서 완전히 익힙니다.',
    '5. 완성된 계란찜을 그릇에 담아 드시면 됩니다.']}]}

In [52]:
output = refing_master_json("계란, 찬밥, 파김치, 장조림")

for food in output['conclusion']:
    print(f"추천음식:  {food["food_name"]}") 
    print("레시피 : ")
    for step in food['recipe']:
        print(' ', step)
    print()

추천음식:  장조림 파김치 계란 볶음밥
레시피 : 
  1. 찬밥을 그릇에 담아 잘 풀어 준비합니다.
  2. 계란을 풀어 팬에 기름을 두르고 스크램블해 반쯤 익혀 따로 둡니다.
  3. 파김치와 장조림을 잘게 다져 준비합니다.
  4. 팬에 기름을 두르고 다진 파김치를 살짝 볶다가 찬밥을 넣고 고루 섞으며 볶습니다.
  5. 미리 익힌 계란과 다진 장조림을 넣고 간장, 소금, 후추로 간을 맞추면서 볶아줍니다.
  6. 모든 재료가 잘 어우러지도록 한 번 더 볶아 불을 끄고 완성합니다.

추천음식:  장조림 계란말이
레시피 : 
  1. 장조림을 작게 잘게 썰어 준비합니다.
  2. 계란을 볼에 깨뜨려 잘 풀고 소금과 후추로 간합니다.
  3. 계란물에 썰어둔 장조림을 넣고 잘 섞습니다.
  4. 달군 프라이팬에 식용유를 두르고 계란물의 일부를 얇게 펴서 부친 후 반쯤 익었을 때 말아줍니다.
  5. 팬 한쪽으로 계란말이를 밀어두고 빈 공간에 계란물을 다시 부어 익히고 말아 겹겹이 쌓아 만듭니다.
  6. 마지막으로 전체를 익혀 계란말이를 완성하고 적당한 크기로 썰어 접시에 담습니다.



## Job Interview 준비

In [55]:
# 채용공고/스펙은 입력받아 면접 질문과 모범답안을 json형태로 반환하는 함수
def job_interview_json(user_input: str, /, *, model='gpt-4.1-mini', temperature=1, top_p=1, max_completion_tokens=2048):
    response = client.chat.completions.create(                 # OpenAI Chat Completions API 호출
        model=model,                                           # 사용할 모델 지정
        messages=[                                             # 대화 메시지 목록
            {
                "role": "system",                              # 시스템 역할: 모델의 페르소나/규칙 정의
                "content": [
                    {
                        "type": "text",                        # 텍스트 타입 메시지
                        "text": """
# Instruction
당신은 20년 경력의 ML/DL 엔지니어이고, 이번 신입개발자 채용의 면접관이다.
주어진 job posting(회사정보)를 바탕으로 신입개발자에 제공할 면접 질문과 모범 답변을 작성하세요.
크게 hard skill과 soft skill/leadership 질문을 구분하여 제시하십시오.

아래 지침을 반드시 준수하세요:

- 면접 질문 및 답변은 반드시 한글로, 그리고 신입 개발자에게 현실적으로 맞도록 작성합니다.
- hard skill(기술 질문)과 soft skill/leadership(소통, 태도, 리더십 등) 영역을 구분해 각각 최소 2개 이상의 예시를 만듭니다.
- 입력 정보가 부족한 경우, 통상적인 산업/포지션 상황에 근거해 논리적으로 추론해 주세요.
- 답변 전체의 output format은 반드시 아래 JSON 형태여야 하며, reasoning > hard_skill > soft_skill_leadership 순서로, 각 영역에 반드시 "질문"과 "모범답변"이 쌍으로 들어갑니다.
- 입력값 미제공 시 [회사정보], [스펙] 등의 placeholder를 사용하세요.

# Output Format

아래 JSON 포맷으로 여는 코드블럭 없이 반드시 출력합니다:

{
  "hard_skill": [
    {
      "question": "[hard skill(기술 중점) 면접 질문]",
      "answer": "[신입개발자 입장에서 모범답변]"
    },
    …
  ],
  "soft_skill_leadership": [
    {
      "question": "[soft skill/leadership(태도, 커뮤니케이션, 성장잠재력 등) 면접 질문]",
      "answer": "[신입개발자 입장에서 모범답변]"
    },
    …
  ]
}

- 각 영역별 면접 질문과 모범답변은 모두 한글로 작성합니다.
- 반드시 하드 스킬, 소프트 스킬/리더십 항목을 구분하여 각 2개 이상 작성합니다(총 4개 이상).
- 예시(Example)처럼 구조, 문장 길이, 구체성, 답변 스타일을 맞추되, 지원자는 신입개발자임을 꼭 반영하세요.

# 예시(Example)

Input:
회사정보: [AI 기업, 제조공정 데이터 분석 솔루션 개발]
스펙: [전자공학 전공, 신입 개발자, Python/Java 가능, 인턴 경험 있음]

Output:

{{
  "hard_skill": [
    {
      "question": "Python 언어를 사용하여 데이터 전처리 경험이 있나요? 예시를 들어 설명해 주세요.",
      "answer": "대학 시절 프로젝트에서 Pandas를 이용해 결측치 처리와 이상치 제거를 해본 경험이 있습니다. 데이터 정제의 중요성을 실제로 체감할 수 있었습니다."
    },
    {
      "question": "제조 데이터와 같이 구조적인 데이터를 다룰 때 주의해야 하는 점은 무엇이라고 생각하나요?",
      "answer": "데이터의 정확한 구조 파악과, 이상치 및 에러를 사전에 점검하는 것이 중요하다고 생각합니다."
    }
  ],
  "soft_skill_leadership": [
    {
      "question": "팀 프로젝트에서 갈등이 있을 때 어떻게 해결해보셨나요?",
      "answer": "인턴 경험 중 팀원과 의견이 달랐을 때 서로의 입장을 경청한 뒤 중간 지점을 찾아 협력한 경험이 있습니다."
    },
    {
      "question": "빠르게 변화하는 환경에서 새로운 기술을 습득했던 경험이 있나요?",
      "answer": "새로운 도구가 필요했던 프로젝트에서 적극적으로 온라인 자료를 찾아 공부하며 습득하였습니다."
    }
  ]
}}

# Notes
- 지원자는 신입개발자, 면접관 페르소나는 20년차 ml/dl 엔지니어임을 모든 답변에 일관되게 반영하세요.
- 모든 콘텍스트와 답변, 지시문, reasoning, 사례, placeholder, Q&A 등은 전체 한글로 작성합니다.
- 포맷(최상단 하드스킬 > 소프트스킬/리더십)과 다국어 혼용 불허, 신입 시점 미반영 등은 모두 미승인 처리 대상입니다.

**Objective Reminder:**
1. '20년차 ml/dl engineer' 페르소나로서, 신입 개발자 대상 면접 Q&A를 회사/직무 정보로부터 reasoning → hard/soft skill/leaderhip별 제시
2. JSON 아웃풋 포맷만 사용 (코드블럭 넣지 않음)                        
                        """
                                                                # 제목 교정 전문가 역할 + 상세 규칙 프롬프트
                    }
                ]
            },
            {
                "role": "user",                                # 사용자 역할: 실제 입력 데이터
                "content": [
                    {
                        "type": "text",                        # 텍스트 입력
                        "text": f"입력: {user_input}"            # 함수 인자로 받은 기사 제목(헤드라인)
                    }
                ]
            }
        ],
        response_format={"type": "json_object"},               # 응답을 순수 json으로 받음
        temperature=temperature,                               # 생성 다양성 조절
        max_completion_tokens=max_completion_tokens,           # 최대 응답 토큰 수
        top_p=top_p,                                           # 확률 질량 기반 샘플링
        frequency_penalty=0,                                   # 반복 패널티 없음
        presence_penalty=0,                                    # 새로운 주제 유도 패널티 없음
        store=False                                            # 서버 저장 비활성화
    )
    return json.loads(response.choices[0].message.content)                 # 모델이 생성한 최종 결과 반환

In [56]:
# 입력된 공고 내용을 기반으로  먄접 질문/모범답안
job_interview_json("""
# Instruction
당신은 20년 경력의 ML/DL 엔지니어이고, 이번 신입개발자 채용의 면접관이다.
주어진 job posting(회사정보)를 바탕으로 신입개발자에 제공할 면접 질문과 모범 답변을 작성하세요.
크게 hard skill과 soft skill/leadership 질문을 구분하여 제시하십시오.

아래 지침을 반드시 준수하세요:

- 면접 질문 및 답변은 반드시 한글로, 그리고 신입 개발자에게 현실적으로 맞도록 작성합니다.
- hard skill(기술 질문)과 soft skill/leadership(소통, 태도, 리더십 등) 영역을 구분해 각각 최소 2개 이상의 예시를 만듭니다.
- 입력 정보가 부족한 경우, 통상적인 산업/포지션 상황에 근거해 논리적으로 추론해 주세요.
- 답변 전체의 output format은 반드시 아래 JSON 형태여야 하며, reasoning > hard_skill > soft_skill_leadership 순서로, 각 영역에 반드시 "질문"과 "모범답변"이 쌍으로 들어갑니다.
- 입력값 미제공 시 [회사정보], [스펙] 등의 placeholder를 사용하세요.

# Output Format

아래 JSON 포맷으로 여는 코드블럭 없이 반드시 출력합니다:

{
  "hard_skill": [
    {
      "question": "[hard skill(기술 중점) 면접 질문]",
      "answer": "[신입개발자 입장에서 모범답변]"
    },
    …
  ],
  "soft_skill_leadership": [
    {
      "question": "[soft skill/leadership(태도, 커뮤니케이션, 성장잠재력 등) 면접 질문]",
      "answer": "[신입개발자 입장에서 모범답변]"
    },
    …
  ]
}

- 각 영역별 면접 질문과 모범답변은 모두 한글로 작성합니다.
- 반드시 하드 스킬, 소프트 스킬/리더십 항목을 구분하여 각 2개 이상 작성합니다(총 4개 이상).
- 예시(Example)처럼 구조, 문장 길이, 구체성, 답변 스타일을 맞추되, 지원자는 신입개발자임을 꼭 반영하세요.

# 예시(Example)

Input:
회사정보: [AI 기업, 제조공정 데이터 분석 솔루션 개발]
스펙: [전자공학 전공, 신입 개발자, Python/Java 가능, 인턴 경험 있음]

Output:

{{
  "hard_skill": [
    {
      "question": "Python 언어를 사용하여 데이터 전처리 경험이 있나요? 예시를 들어 설명해 주세요.",
      "answer": "대학 시절 프로젝트에서 Pandas를 이용해 결측치 처리와 이상치 제거를 해본 경험이 있습니다. 데이터 정제의 중요성을 실제로 체감할 수 있었습니다."
    },
    {
      "question": "제조 데이터와 같이 구조적인 데이터를 다룰 때 주의해야 하는 점은 무엇이라고 생각하나요?",
      "answer": "데이터의 정확한 구조 파악과, 이상치 및 에러를 사전에 점검하는 것이 중요하다고 생각합니다."
    }
  ],
  "soft_skill_leadership": [
    {
      "question": "팀 프로젝트에서 갈등이 있을 때 어떻게 해결해보셨나요?",
      "answer": "인턴 경험 중 팀원과 의견이 달랐을 때 서로의 입장을 경청한 뒤 중간 지점을 찾아 협력한 경험이 있습니다."
    },
    {
      "question": "빠르게 변화하는 환경에서 새로운 기술을 습득했던 경험이 있나요?",
      "answer": "새로운 도구가 필요했던 프로젝트에서 적극적으로 온라인 자료를 찾아 공부하며 습득하였습니다."
    }
  ]
}}

# Notes
- 지원자는 신입개발자, 면접관 페르소나는 20년차 ml/dl 엔지니어임을 모든 답변에 일관되게 반영하세요.
- 모든 콘텍스트와 답변, 지시문, reasoning, 사례, placeholder, Q&A 등은 전체 한글로 작성합니다.
- 포맷(최상단 하드스킬 > 소프트스킬/리더십)과 다국어 혼용 불허, 신입 시점 미반영 등은 모두 미승인 처리 대상입니다.

**Objective Reminder:**
1. '20년차 ml/dl engineer' 페르소나로서, 신입 개발자 대상 면접 Q&A를 회사/직무 정보로부터 reasoning → hard/soft skill/leaderhip별 제시
2. JSON 아웃풋 포맷만 사용 (코드블럭 넣지 않음)
                   
                   """)

{'hard_skill': [{'question': '머신러닝 모델을 학습시킬 때 과적합(overfitting)이란 무엇이며, 이를 방지하기 위해 어떤 방법을 사용할 수 있나요?',
   'answer': '과적합은 모델이 학습 데이터에 너무 맞춰져서 새로운 데이터에 일반화되지 못하는 현상입니다. 이를 방지하기 위해 교차 검증, 정규화(L2, L1), 드롭아웃, 데이터 증강 등을 활용하며, 적절한 모델 복잡도를 선택하는 것도 중요하다고 알고 있습니다.'},
  {'question': 'Python을 사용하여 간단한 데이터 전처리 작업을 수행해본 경험이 있나요? 어떤 라이브러리를 사용했고, 어떤 작업을 했는지 설명해 주세요.',
   'answer': '대학 프로젝트에서 Pandas를 사용해 결측치를 처리하고, 필요 없는 열을 제거하는 전처리 작업을 해보았습니다. 또한, 간단하게 그룹화나 정렬도 해보고 데이터의 통계값을 확인하였습니다.'}],
 'soft_skill_leadership': [{'question': '팀 프로젝트에서 의견 충돌이 발생했을 때, 어떻게 대처하셨나요?',
   'answer': '인턴십 동안 팀원이 다른 의견을 제시했을 때, 먼저 상대방 의견을 경청하고 왜 그런 생각을 했는지 질문하며 이해하려 노력했습니다. 이후 제 의견도 논리적으로 설명하며 서로 타협점을 찾아 협업을 원활히 진행했던 경험이 있습니다.'},
  {'question': '새로운 기술이나 도구를 빠르게 학습해야 할 때 본인은 어떤 방법으로 접근하나요?',
   'answer': '필요한 기술과 관련된 공식 문서와 튜토리얼 영상을 참고하고, 실제로 코드를 작성해보며 이해도를 높입니다. 또한, 온라인 커뮤니티나 질문 사이트에서 유사 사례를 찾아보며 실무에 적용할 수 있도록 노력합니다.'}]}

In [57]:
job_interview_json("""
## Job Descriptions

주요 업무 내용 안내

### 코딕스 개발팀 소개말

코딕스 개발팀은 콘텐츠 플랫폼 테스트북을 중심으로 일하고 있습니다. 사용자들이 더 편하게 서비스를 이용하도록 돕고있어요. 테스트북 뿐만 아니라 웹을 기반으로 한 다양한 서비스를 개발합니다.
기존에 사용하던 도구 이외에 새로운 Framework나 기술에도 관심이 많으며 적극적으로 검토하고 도입하기 위해 노력해요. 물론 개인의 역량과 커리어 증진에도 힘쓰고 있습니다.

---

## AI 개발자 주요 업무 내용 안내

### AI 개발자 (AI Developer)

AI 기술을 이용한 교육용 애플리케이션 프로젝트를 개발(ML/DL/Generative AI)하고,
Python 기반 API 서버 개발 및 Kubernetes 기반 배포 환경 구성합니다.

---

### 기술 및 자격 요건

* Python 프로그래밍 (FastAPI, Django, Flask 등) 경험이 있는 사람
* NLP 프로젝트 또는 LLM + RAG + VectorDB 기반 개발 경험이 있는 사람
* RESTful API 설계 및 Kubernetes 기반 배포 경험이 있는 사람
* 업무 커뮤니케이션에 대한 소통 능력이 뛰어난 분

---

### 우대 사항

* 바이브 코딩 경험을 보유하신 분
* 생성형 AI 관련 프로젝트 경험이 있는 사람
* 각종 협업 도구에 익숙하고 새로운 사용에 적극적인 사람
""")

{'hard_skill': [{'question': 'Python 기반 웹 프레임워크 중 FastAPI, Django, Flask 중 하나를 사용해 본 경험이 있나요? 사용하면서 어려웠던 점과 해결 방법을 설명해 주세요.',
   'answer': '대학교 프로젝트에서 Flask를 사용해 간단한 웹 API를 구현해본 경험이 있습니다. 초기에는 라우팅과 요청 처리 방식이 익숙하지 않아 어려웠지만, 공식 문서와 예제 코드를 참고하며 이해를 높였고, 특히 API 문서 자동화 기능이 편리하다는 것을 알게 되었습니다.'},
  {'question': 'NLP 또는 기본적인 머신러닝/딥러닝 모델을 학습해 보셨나요? 간단한 예를 들어 어떤 프로젝트였고 어떤 문제를 해결했는지 설명해 주세요.',
   'answer': '학부 과정에서 자연어 처리 기법을 적용해 영화 리뷰 감성 분류 프로젝트를 진행한 적이 있습니다. 텍스트 전처리부터 TF-IDF 벡터화, 간단한 Logistic Regression 모델을 사용해 긍정과 부정을 분류했습니다. 모델 성능 향상을 위해 데이터 전처리와 하이퍼파라미터 튜닝에 노력했습니다.'},
  {'question': 'RESTful API를 설계할 때 어떤 원칙을 중요하게 생각하나요? 그리고 Kubernetes 배포 환경에 대해서 아는 바를 설명해 주세요.',
   'answer': 'RESTful API 설계 시에는 자원을 명확히 정의하고 HTTP 메서드를 적절히 사용하는 것이 중요하다고 생각합니다. 예를 들어 GET은 조회, POST는 생성, PUT/PATCH는 수정, DELETE는 삭제에 사용합니다. Kubernetes는 컨테이너화된 애플리케이션을 자동으로 배포하고 관리해주는 오케스트레이션 도구로, 신입 때는 기본 개념과 간단한 배포 매니페스트 작성 정도를 공부했습니다.'},
  {'question': 'Python을 사용하여 간단한 서버를 만들고 Kubernetes 클러스터에 배포한다고 가정할 때 필요한 주요 단계를 설명해 주세요.',
  

In [60]:
output_interview = job_interview_json("""
ENSSEL,
AI 사업부,
AI(인공지능) 개발자,
---

모집 분야,
모집 부문,
AI 사업부
AI(인공지능) 개발자

업무 내용,
인공지능 알고리즘 개발 및 연구,
인공지능 서비스 솔루션 개발,

  
Vision
Time Series,
NLP,
,
데이터 처리 및 분석,
언어 : Python,

자격 요건 및 우대사항,
대졸 이상,
전반적인 AI 기술에 대한 이해가 있으신 분,

  
Vision / TimeSeries / NLP
딥러닝 모델에 대한 이해 및 학습 경험,
,
딥러닝 프레임워크 (Pytorch, Tensorflow 등) 사용,
데이터를 분석하거나 모델링을 통해 결과를 도출해본 경험이 있으신 분,
*경력자의 경우, PMP 자격증 소지자 우대,

모집 인원,
0명
""")

In [65]:
for hard in output_interview["hard_skill"]:
    print("question:", hard["question"])
    print("answer:", hard["answer"])
    print()

question: Python을 활용하여 인공지능 모델을 학습시켜본 경험이 있나요? 구체적인 프로젝트나 사용한 라이브러리를 설명해 주세요.
answer: 대학에서 딥러닝 과목 프로젝트를 진행하며 Python과 PyTorch를 사용해 간단한 이미지 분류 모델을 학습시켜본 경험이 있습니다. 데이터 전처리부터 모델 설계, 학습, 평가까지 전 과정을 체험하며 딥러닝 프레임워크 사용법에 익숙해졌습니다.

question: 시계열(Time Series) 데이터 분석에 대해 알고 있나요? 시계열 데이터의 특징과 이를 다룰 때 중요한 점은 무엇인지 설명해 주세요.
answer: 시계열 데이터는 시간의 흐름에 따라 변하는 데이터로, 시점 간의 연속성과 계절성 등이 중요한 특징입니다. 따라서 데이터를 분석할 때는 시계열의 추세, 계절성, 노이즈를 분리하고, 적절한 모델을 선택하는 것이 중요하다고 생각합니다.



In [66]:
for  soft in output_interview["soft_skill_leadership"]:
    print("question:", soft["question"])
    print("answer:", soft["answer"])
    print()

question: 팀 프로젝트에서 의견 충돌이 있었을 때 어떻게 대처했는지 경험을 말씀해 주세요.
answer: 대학 팀 프로젝트 중 모델 선택에 대한 의견 차이가 있었을 때, 서로의 의견과 근거를 경청하고 장단점을 비교하는 시간을 가졌습니다. 이를 통해 모두가 이해할 수 있는 최선의 방안을 찾았고, 협력하여 프로젝트를 성공적으로 마무리할 수 있었습니다.

question: 새로운 AI 기술이나 딥러닝 프레임워크를 빠르게 습득하기 위해 평소 어떤 노력을 해왔나요?
answer: 저는 최신 연구 논문과 온라인 강의를 통해 꾸준히 공부하고, 개인 프로젝트에 적용해보면서 이해도를 높였습니다. 또한, 모르는 부분은 커뮤니티나 스터디에서 질문하며 실력을 강화하고자 노력하고 있습니다.

